# Are High School Equivalency Test Passing Rates Correlated to Length of Test (short vs long version)?

In [1]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from sqlalchemy import create_engine
from datetime import datetime 
import matplotlib.pyplot as plt

In [2]:
# Read and Store XLSX file into DataFrame
NWEA_Data_df = pd.read_excel('Resources/NWEAGrowthReportYear2019.xlsx', sheet_name='2019')
print("Column headings:")
print(NWEA_Data_df.columns)

Column headings:
Index(['Student ID', 'Student Last', 'Student First', 'Term Tested', 'Subject',
       'Test RIT Score', 'Rapid-Guessing %', 'Assessment Name',
       'Math: Geometry', 'Math: Operations and Algebraic Thinking',
       'Math: Statistics and Probability',
       'Math: The Real and Complex Number Systems',
       'Reading: Informational Text', 'Reading: Literature',
       'Reading: Vocabulary Acquisition and Use',
       'Language: Language: Understand, Edit for Grammar, Usage',
       'Language: Language: Understand, Edit Mechanics',
       'Language: Writing: Plan, Organize, Develop, Revise, Research'],
      dtype='object')


In [3]:
# Remove Inactive CMs
NWEA1_df = NWEA_Data_df[~NWEA_Data_df["Term Tested"].str.contains("Summer", na=False)]
NWEA2_df = NWEA1_df[~NWEA1_df["Term Tested"].str.contains("Fall", na=False)]
NWEA3_df = NWEA2_df[~NWEA2_df["Term Tested"].str.contains("Winter", na=False)]
NWEA_df = NWEA3_df[~NWEA3_df["Term Tested"].str.contains("Spring", na=False)]

print(NWEA_df)

     Student ID    Student Last Student First          Term Tested    Subject  \
0        2262.0         Abraham        Jelani  2019-01-09 00:00:00  Language    
1        2262.0         Abraham        Jelani  2019-04-11 00:00:00   Language   
2        2262.0         Abraham        Jelani  2019-07-11 00:00:00   Language   
3        2262.0         Abraham        Jelani  2019-08-29 00:00:00   Language   
4        2262.0         Abraham        Jelani  2019-12-09 00:00:00   Language   
5        4770.0  Acosta Delgado      Estefany  2019-04-08 00:00:00   Language   
6        4770.0  Acosta Delgado      Estefany  2019-08-29 00:00:00   Language   
7        4770.0  Acosta Delgado      Estefany  2019-12-09 00:00:00   Language   
10       2375.0           Akins       Madison  2019-01-17 00:00:00   Language   
11       2375.0           Akins       Madison  2019-04-02 00:00:00   Language   
12       2066.0           Allen         Imani  2019-07-11 00:00:00   Language   
13       2066.0           Al

In [4]:
# Convert 'Test Date' from objects to datetime
NWEA_df[["Test Date"]] = NWEA_df[["Term Tested"]].astype('datetime64[ns]')
NWEA_df = NWEA_df.drop('Term Tested', axis=1)

NWEA_df.head()

C:\Users\314ca\Anaconda3\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,Student ID,Student Last,Student First,Subject,Test RIT Score,Rapid-Guessing %,Assessment Name,Math: Geometry,Math: Operations and Algebraic Thinking,Math: Statistics and Probability,Math: The Real and Complex Number Systems,Reading: Informational Text,Reading: Literature,Reading: Vocabulary Acquisition and Use,"Language: Language: Understand, Edit for Grammar, Usage","Language: Language: Understand, Edit Mechanics","Language: Writing: Plan, Organize, Develop, Revise, Research",Test Date
0,2262.0,Abraham,Jelani,Language,201.0,4.0,Growth (Long),NaN,NaN,NaN,NaN,NaN,NaN,NaN,201-210,201-210,191-200,2019-01-09
1,2262.0,Abraham,Jelani,Language,215.0,NaN,Screening (Short),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-11
2,2262.0,Abraham,Jelani,Language,193.0,4.0,Growth (Long),NaN,NaN,NaN,NaN,NaN,NaN,NaN,201-210,181-190,181-190,2019-07-11
3,2262.0,Abraham,Jelani,Language,198.0,NaN,Screening (Short),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-08-29
4,2262.0,Abraham,Jelani,Language,217.0,NaN,Screening (Short),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-09


In [5]:
# Using .rename(columns={}) in order to rename columns
renamed_df = NWEA_df.rename(columns={"Assessment Name":"Version", "Math: Operations and Algebraic Thinking": "Math: Algebra", 
                                     "Math: The Real and Complex Number Systems": "Math: Numbers", "Reading: Vocabulary Acquisition and Use":"Reading: Vocab",
                                    "Language: Language: Understand, Edit for Grammar, Usage":"Language: Grammar", "Language: Language: Understand, Edit Mechanics": "Language: Mechanics",
                                    "Language: Writing: Plan, Organize, Develop, Revise, Research": "Language: Writing"})

renamed_df.head()

,Student ID,Student Last,Student First,Subject,Test RIT Score,Rapid-Guessing %,Version,Math: Geometry,Math: Algebra,Math: Statistics and Probability,Math: Numbers,Reading: Informational Text,Reading: Literature,Reading: Vocab,Language: Grammar,Language: Mechanics,Language: Writing,Test Date
0,2262.0,Abraham,Jelani,Language,201.0,4.0,Growth (Long),NaN,NaN,NaN,NaN,NaN,NaN,NaN,201-210,201-210,191-200,2019-01-09
1,2262.0,Abraham,Jelani,Language,215.0,NaN,Screening (Short),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-11
2,2262.0,Abraham,Jelani,Language,193.0,4.0,Growth (Long),NaN,NaN,NaN,NaN,NaN,NaN,NaN,201-210,181-190,181-190,2019-07-11
3,2262.0,Abraham,Jelani,Language,198.0,NaN,Screening (Short),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-08-29
4,2262.0,Abraham,Jelani,Language,217.0,NaN,Screening (Short),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-09


In [6]:
# Reorganizing the columns using double brackets
organized_df = renamed_df[["Student ID","Student Last","Student First", "Test Date", "Subject", "Version", "Test RIT Score", "Rapid-Guessing %"]]
organized_df.head()

,Student ID,Student Last,Student First,Test Date,Subject,Version,Test RIT Score,Rapid-Guessing %
0,2262.0,Abraham,Jelani,2019-01-09,Language,Growth (Long),201.0,4.0
1,2262.0,Abraham,Jelani,2019-04-11,Language,Screening (Short),215.0,NaN
2,2262.0,Abraham,Jelani,2019-07-11,Language,Growth (Long),193.0,4.0
3,2262.0,Abraham,Jelani,2019-08-29,Language,Screening (Short),198.0,NaN
4,2262.0,Abraham,Jelani,2019-12-09,Language,Screening (Short),217.0,NaN


In [10]:
# Parse out each subject of test
Language_df = organized_df[organized_df["Subject"].str.contains("Language", na=False)]
Math_df = organized_df[organized_df["Subject"].str.contains("Math", na=False)]
Reading_df = organized_df[organized_df["Subject"].str.contains("Reading", na=False)]

Math_df.head()

,Student ID,Student Last,Student First,Test Date,Subject,Version,Test RIT Score,Rapid-Guessing %
192,2262.0,Abraham,Jelani,2019-01-11,Math,Growth (Long),200.0,0.0
193,2262.0,Abraham,Jelani,2019-07-10,Math,Growth (Long),206.0,0.0
194,2263.0,Abraham,Jelani,2019-10-10,Math,Screening (Short),210.0,NaN
195,4770.0,Acosta Delgado,Estefany,2019-06-03,Math,Screening (Short),210.0,NaN
196,4770.0,Acosta Delgado,Estefany,2019-08-27,Math,Screening (Short),214.0,NaN


In [51]:
# Math Tests Given Analysis
Total_Math_Tests = len(Math_df["Student ID"])
Total_Math_Tests

Total_MStudents_Tested = len(Math_df["Student ID"].unique())
Total_MStudents_Tested

Short_MVersions = Math_df[Math_df["Version"].str.contains("Screening")]
Short_MVersions_Count = len(Short_MVersions)
Short_MVersions_Count

Long_MVersions = Math_df[Math_df["Version"].str.contains("Growth")]
Long_MVersions_Count = len(Long_MVersions)
Long_MVersions_Count


Math_summary_table = pd.DataFrame({
    "Total Math Tests": [Total_Math_Tests],
    "Total Students Tested": [Total_MStudents_Tested],
    "Short Versions": [Short_MVersions_Count],
    "Long Versions": [Long_MVersions_Count],
    "Short Version as % of Total Tests": [round(((Short_MVersions_Count/Total_Math_Tests) * 100), 1)]
})
Math_summary_table

,Total Math Tests,Total Students Tested,Short Versions,Long Versions,Short Version as % of Total Tests
0,140,74,46,94,32.9


In [55]:
# Language Tests Given Analysis
Total_Lang_Tests = len(Language_df["Student ID"])
Total_Lang_Tests

Total_LStudents_Tested = len(Language_df["Student ID"].unique())
Total_LStudents_Tested

Short_LVersions = Language_df[Language_df["Version"].str.contains("Screening")]
Short_LVersions_Count = len(Short_LVersions)
Short_LVersions_Count

Long_LVersions = Language_df[Language_df["Version"].str.contains("Growth")]
Long_LVersions_Count = len(Long_LVersions)
Long_LVersions_Count


Lang_summary_table = pd.DataFrame({
    "Total Language Tests": [Total_Lang_Tests],
    "Total Students Tested": [Total_LStudents_Tested],
    "Short Versions": [Short_LVersions_Count],
    "Long Versions": [Long_LVersions_Count],
    "Short Version as % of Total Tests": [round(((Short_LVersions_Count/Total_Lang_Tests) * 100), 1)]
})
Lang_summary_table

,Total Language Tests,Total Students Tested,Short Versions,Long Versions,Short Version as % of Total Tests
0,153,82,54,99,35.3


In [56]:
# Reading Tests Given Analysis
Total_Reading_Tests = len(Reading_df["Student ID"])
Total_Reading_Tests

Total_RStudents_Tested = len(Reading_df["Student ID"].unique())
Total_RStudents_Tested

Short_RVersions = Reading_df[Reading_df["Version"].str.contains("Screening")]
Short_RVersions_Count = len(Short_RVersions)
Short_RVersions_Count

Long_RVersions = Reading_df[Reading_df["Version"].str.contains("Growth")]
Long_RVersions_Count = len(Long_RVersions)
Long_RVersions_Count


Reading_summary_table = pd.DataFrame({
    "Total Reading Tests": [Total_Reading_Tests],
    "Total Students Tested": [Total_RStudents_Tested],
    "Short Versions": [Short_RVersions_Count],
    "Long Versions": [Long_RVersions_Count],
    "Short Version as % of Total Tests": [round(((Short_RVersions_Count/Total_Reading_Tests) * 100), 1)]
})
Reading_summary_table

,Total Reading Tests,Total Students Tested,Short Versions,Long Versions,Short Version as % of Total Tests
0,138,70,66,72,47.8


In [ ]:
# Find Average Score for Short vs. Long Test
